# INTERNET SALES FACT

In [1]:
import pandas as pd
from connection_script import connect_databases
from datetime import datetime

In [2]:
db_op, db_etl = connect_databases()

## EXTRACT

In [3]:
# Extract required data
# Sales online orders
sales_order = pd.read_sql_query("""
        SELECT *,[dbo].[ufnGetProductStandardCost](t2.ProductID, t1.OrderDate) AS ProductStandardCost
        FROM Sales.SalesOrderHeader AS t1
        INNER JOIN Sales.SalesOrderDetail AS t2
        ON t1.SalesOrderID = t2.SalesOrderID
        WHERE t1.OnlineOrderFlag = 1;
        """, db_op)

In [4]:
# Load required dimensions
dim_product =  pd.read_sql_query('SELECT * FROM "DimProduct";', db_etl)
dim_product

,ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,StandardCost,FinishedGoodsFlag,Color,SafetyStockLevel,...,Class,Style,ModelName,LargePhoto,EnglishDescription,FrenchDescription,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription
0,1,AR-5381,NaN,None,None,Adjustable Race,0.0000,False,None,1000,...,None,None,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None
1,2,BA-8327,NaN,None,None,Bearing Ball,0.0000,False,None,1000,...,None,None,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None
2,3,BE-2349,NaN,None,None,BB Ball Bearing,0.0000,False,None,800,...,None,None,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None
3,4,BE-2908,NaN,None,None,Headset Ball Bearings,0.0000,False,None,800,...,None,None,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None
4,5,BL-2036,NaN,None,None,Blade,0.0000,False,None,800,...,None,None,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,601,BB-7421,5.0,G,None,LL Bottom Bracket,23.9716,True,None,500,...,L,None,LL Bottom Bracket,\x474946383961f0009500f70000000000800000008000...,Chromoly steel.,Acier chromé.,铬钢。,الفولاذ الكرومولي,פלדת כרומולי (כרום-מולובדניום),โลหะโครโมลี
602,602,BB-8107,5.0,G,None,ML Bottom Bracket,44.9506,True,None,500,...,M,None,ML Bottom Bracket,\x474946383961f0009500f70000000000800000008000...,Aluminum alloy cups; large diameter spindle.,Cuvettes en alliage d'aluminium ; axe de grand...,铝合金车圈；大直径脚蹬轴。,فناجين من سبيكة الألومنيوم؛ ذات محور دوران كبي...,כיסויים מסגסוגת אלומיניום; ציר רחב-קוטר.,ดุมอลูมิเนียมอัลลอยด์ แกนเพลาขนาดใหญ่
603,604,BK-R19B-44,2.0,LB,CM,"Road-750 Black, 44",343.6496,True,Black,100,...,L,U,Road-750,\x474946383961f0009500f700008a7677f7f7f7edecec...,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,入门级成人自行车；确保越野旅行或公路骑乘的舒适。快拆式车毂和轮缘。,إنها دراجة مناسبة للمبتدئين من البالغين؛ فهي ت...,"אופני מבוגרים למתחילים; מציעים רכיבה נוחה ""מחו...",จักรยานระดับเริ่มต้นสำหรับผู้ใหญ่ ให้ความสบายใ...
604,605,BK-R19B-48,2.0,LB,CM,"Road-750 Black, 48",343.6496,True,Black,100,...,L,U,Road-750,\x474946383961f0009500f700008a7677f7f7f7edecec...,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,入门级成人自行车；确保越野旅行或公路骑乘的舒适。快拆式车毂和轮缘。,إنها دراجة مناسبة للمبتدئين من البالغين؛ فهي ت...,"אופני מבוגרים למתחילים; מציעים רכיבה נוחה ""מחו...",จักรยานระดับเริ่มต้นสำหรับผู้ใหญ่ ให้ความสบายใ...


In [5]:
operational_products = pd.read_sql_query('SELECT ProductID, ProductNumber FROM [Production].[Product];', db_op)

In [6]:
dim_product = dim_product.merge(operational_products, left_on="ProductAlternateKey", right_on="ProductNumber")
dim_product = dim_product.drop_duplicates(subset=["ProductID"])

In [7]:
dim_product

,ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,StandardCost,FinishedGoodsFlag,Color,SafetyStockLevel,...,ModelName,LargePhoto,EnglishDescription,FrenchDescription,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,ProductID,ProductNumber
0,1,AR-5381,NaN,None,None,Adjustable Race,0.0000,False,None,1000,...,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None,1,AR-5381
1,2,BA-8327,NaN,None,None,Bearing Ball,0.0000,False,None,1000,...,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None,2,BA-8327
2,3,BE-2349,NaN,None,None,BB Ball Bearing,0.0000,False,None,800,...,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None,3,BE-2349
3,4,BE-2908,NaN,None,None,Headset Ball Bearings,0.0000,False,None,800,...,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None,4,BE-2908
4,5,BL-2036,NaN,None,None,Blade,0.0000,False,None,800,...,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None,316,BL-2036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,601,BB-7421,5.0,G,None,LL Bottom Bracket,23.9716,True,None,500,...,LL Bottom Bracket,\x474946383961f0009500f70000000000800000008000...,Chromoly steel.,Acier chromé.,铬钢。,الفولاذ الكرومولي,פלדת כרומולי (כרום-מולובדניום),โลหะโครโมลี,994,BB-7421
602,602,BB-8107,5.0,G,None,ML Bottom Bracket,44.9506,True,None,500,...,ML Bottom Bracket,\x474946383961f0009500f70000000000800000008000...,Aluminum alloy cups; large diameter spindle.,Cuvettes en alliage d'aluminium ; axe de grand...,铝合金车圈；大直径脚蹬轴。,فناجين من سبيكة الألومنيوم؛ ذات محور دوران كبي...,כיסויים מסגסוגת אלומיניום; ציר רחב-קוטר.,ดุมอลูมิเนียมอัลลอยด์ แกนเพลาขนาดใหญ่,995,BB-8107
603,604,BK-R19B-44,2.0,LB,CM,"Road-750 Black, 44",343.6496,True,Black,100,...,Road-750,\x474946383961f0009500f700008a7677f7f7f7edecec...,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,入门级成人自行车；确保越野旅行或公路骑乘的舒适。快拆式车毂和轮缘。,إنها دراجة مناسبة للمبتدئين من البالغين؛ فهي ت...,"אופני מבוגרים למתחילים; מציעים רכיבה נוחה ""מחו...",จักรยานระดับเริ่มต้นสำหรับผู้ใหญ่ ให้ความสบายใ...,997,BK-R19B-44
604,605,BK-R19B-48,2.0,LB,CM,"Road-750 Black, 48",343.6496,True,Black,100,...,Road-750,\x474946383961f0009500f700008a7677f7f7f7edecec...,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,入门级成人自行车；确保越野旅行或公路骑乘的舒适。快拆式车毂和轮缘。,إنها دراجة مناسبة للمبتدئين من البالغين؛ فهي ت...,"אופני מבוגרים למתחילים; מציעים רכיבה נוחה ""מחו...",จักรยานระดับเริ่มต้นสำหรับผู้ใหญ่ ให้ความสบายใ...,998,BK-R19B-48


In [8]:
dim_customer = pd.read_sql_query('SELECT * FROM "DimCustomer";', db_etl)
dim_customer

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,...,TotalChildren,NumberChildrenAtHome,EnglishEducation,EnglishOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase
0,11000,210,AW00011000,Mr.,David,R.,Robinett,False,1961-02-23,M,...,4.0,0.0,Graduate Degree,Clerical,True,0.0,Pappelallee 6667,None,238-555-0100,2003-09-01
1,11001,37,AW00011001,Ms.,Rebecca,A.,Robinson,False,1965-06-11,M,...,3.0,3.0,Bachelors,Professional,True,1.0,1861 Chinquapin Ct,None,648-555-0100,2004-06-05
2,11002,34,AW00011002,Ms.,Dorothy,B.,Robinson,False,1954-09-23,S,...,2.0,0.0,Partial College,Skilled Manual,False,2.0,4693 Mills Dr.,None,423-555-0100,2002-04-07
3,11003,244,AW00011003,Ms.,Carol Ann,F.,Rockne,False,1943-07-15,M,...,1.0,0.0,Bachelors,Clerical,True,0.0,1312 Skycrest Drive,None,439-555-0100,2001-10-27
4,11004,22,AW00011004,Mr.,Scott,M.,Rodgers,False,1968-05-15,M,...,2.0,2.0,Bachelors,Professional,True,1.0,9860 Brookview Drive,None,989-555-0100,2002-04-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18479,29479,272,AW00029479,None,Crystal,None,Guo,False,1974-11-23,S,...,0.0,0.0,Partial High School,Manual,True,2.0,988 Mt. Everest Court,None,1 (11) 500 555-0171,2004-04-19
18480,29480,380,AW00029480,None,Isabella,F,Richardson,False,1961-06-12,M,...,1.0,0.0,Partial College,Skilled Manual,True,1.0,7413 Alpine Drive,None,910-555-0166,2003-08-30
18481,29481,633,AW00029481,None,Crystal,S,He,False,1940-04-05,M,...,3.0,0.0,Bachelors,Management,True,2.0,4764 East Avenue,None,813-555-0148,2004-04-12
18482,29482,145,AW00029482,None,Crystal,None,Zheng,False,1975-07-25,S,...,0.0,0.0,Partial College,Clerical,True,1.0,"34334, rue Jean Mermoz",None,1 (11) 500 555-0171,2004-02-15


In [9]:
dim_promotion = pd.read_sql_query('SELECT * FROM "DimPromotion";', db_etl)
dim_promotion

,PromotionKey,PromotionAlternateKey,EnglishPromotionName,DiscountPct,EnglishPromotionType,EnglishPromotionCategory,StartDate,EndDate,MinQty,MaxQty
0,1,1,No Discount,0.00,No Discount,No Discount,2011-05-01,2014-11-30,0,NaN
1,2,2,Volume Discount 11 to 14,0.02,Volume Discount,Reseller,2011-05-31,2014-05-30,11,14.0
2,3,3,Volume Discount 15 to 24,0.05,Volume Discount,Reseller,2011-05-31,2014-05-30,15,24.0
3,4,4,Volume Discount 25 to 40,0.10,Volume Discount,Reseller,2011-05-31,2014-05-30,25,40.0
4,5,5,Volume Discount 41 to 60,0.15,Volume Discount,Reseller,2011-05-31,2014-05-30,41,60.0
5,6,6,Volume Discount over 60,0.20,Volume Discount,Reseller,2011-05-31,2014-05-30,61,NaN
6,7,7,Mountain-100 Clearance Sale,0.35,Discontinued Product,Reseller,2012-04-13,2012-05-29,0,NaN
7,8,8,Sport Helmet Discount-2002,0.10,Seasonal Discount,Reseller,2012-05-30,2012-06-29,0,NaN
8,9,9,Road-650 Overstock,0.30,Excess Inventory,Reseller,2012-05-30,2012-07-30,0,NaN
9,10,10,Mountain Tire Sale,0.50,Excess Inventory,Customer,2013-05-14,2013-07-29,0,NaN


In [10]:
dim_sales_territory = pd.read_sql_query('SELECT * FROM "DimSalesTerritory";', db_etl)
dim_sales_territory

,SalesTerritoryKey,SalesTerritoryAlternateKey,SalesTerritoryRegion,SalesTerritoryCountry,SalesTerritoryGroup
0,1,1,Northwest,United States,North America
1,2,2,Northeast,United States,North America
2,3,3,Central,United States,North America
3,4,4,Southwest,United States,North America
4,5,5,Southeast,United States,North America
5,6,6,Canada,Canada,North America
6,7,7,France,France,Europe
7,8,8,Germany,Germany,Europe
8,9,9,Australia,Australia,Pacific
9,10,10,United Kingdom,United Kingdom,Europe


In [11]:
dim_dates = pd.read_sql_query('SELECT * FROM "DimDate";', db_etl)
dim_dates

,DateKey,FullDateAlternateKey,DayNumberOfWeek,EnglishDayNameOfWeek,SpanishDayNameOfWeek,FrenchDayNameOfWeek,DayNumberOfMonth,WeekNumberOfYear,EnglishMonthName,SpanishMonthName,FrenchMonthName,MonthNumberOfYear,CalendarQuarter,CalendarYear,CalendarSemester,FiscalQuarter,FiscalYear,FiscalSemester
0,20050101,2005-01-01,7,Saturday,Sábado,Samedi,1,0,January,Enero,Janvier,1,1,2005,1,3,2005,2
1,20050102,2005-01-02,1,Sunday,Domingo,Dimanche,2,1,January,Enero,Janvier,1,1,2005,1,3,2005,2
2,20050103,2005-01-03,2,Monday,Lunes,Lundi,3,1,January,Enero,Janvier,1,1,2005,1,3,2005,2
3,20050104,2005-01-04,3,Tuesday,Martes,Mardi,4,1,January,Enero,Janvier,1,1,2005,1,3,2005,2
4,20050105,2005-01-05,4,Wednesday,Miércoles,Mercredi,5,1,January,Enero,Janvier,1,1,2005,1,3,2005,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,20141227,2014-12-27,7,Saturday,Sábado,Samedi,361,51,December,Diciembre,Décembre,12,4,2014,2,2,2015,1
3648,20141228,2014-12-28,1,Sunday,Domingo,Dimanche,362,52,December,Diciembre,Décembre,12,4,2014,2,2,2015,1
3649,20141229,2014-12-29,2,Monday,Lunes,Lundi,363,52,December,Diciembre,Décembre,12,4,2014,2,2,2015,1
3650,20141230,2014-12-30,3,Tuesday,Martes,Mardi,364,52,December,Diciembre,Décembre,12,4,2014,2,2,2015,1


In [12]:
dim_currency = pd.read_sql_query('SELECT * FROM "DimCurrency";', db_etl)
dim_currency

,CurrencyKey,CurrencyAlternateKey,CurrencyName
0,1,AFA,Afghani
1,2,DZD,Algerian Dinar
2,3,ARS,Argentine Peso
3,4,AMD,Armenian Dram
4,5,AWG,Aruban Guilder
...,...,...,...
100,101,KRW,Won
101,102,JPY,Yen
102,103,CNY,Yuan Renminbi
103,104,ZWD,Zimbabwe Dollar


In [13]:
currency_rate = pd.read_sql_query('SELECT CurrencyRateID, ToCurrencyCode FROM [Sales].[CurrencyRate]', db_op)

In [14]:
sales_order = sales_order.merge(currency_rate, on="CurrencyRateID", how="left")
sales_order["ToCurrencyCode"] = sales_order["ToCurrencyCode"].apply(lambda x: 'USD' if pd.isna(x) else x )

In [15]:
sales_order = sales_order.merge(dim_currency, left_on="ToCurrencyCode", right_on="CurrencyAlternateKey")

In [16]:
sales_order = sales_order.drop(columns=["ToCurrencyCode", "CurrencyAlternateKey", "CurrencyRateID"])

In [17]:
sales_order["OrderDate"] = sales_order["OrderDate"].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))
sales_order["DueDate"] = sales_order["DueDate"].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))
sales_order["ShipDate"] = sales_order["ShipDate"].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))

## TRANSFORM

In [18]:
dim_product

,ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,StandardCost,FinishedGoodsFlag,Color,SafetyStockLevel,...,ModelName,LargePhoto,EnglishDescription,FrenchDescription,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,ProductID,ProductNumber
0,1,AR-5381,NaN,None,None,Adjustable Race,0.0000,False,None,1000,...,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None,1,AR-5381
1,2,BA-8327,NaN,None,None,Bearing Ball,0.0000,False,None,1000,...,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None,2,BA-8327
2,3,BE-2349,NaN,None,None,BB Ball Bearing,0.0000,False,None,800,...,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None,3,BE-2349
3,4,BE-2908,NaN,None,None,Headset Ball Bearings,0.0000,False,None,800,...,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None,4,BE-2908
4,5,BL-2036,NaN,None,None,Blade,0.0000,False,None,800,...,None,\x474946383961f0009500f70000000000800000008000...,None,None,None,None,None,None,316,BL-2036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,601,BB-7421,5.0,G,None,LL Bottom Bracket,23.9716,True,None,500,...,LL Bottom Bracket,\x474946383961f0009500f70000000000800000008000...,Chromoly steel.,Acier chromé.,铬钢。,الفولاذ الكرومولي,פלדת כרומולי (כרום-מולובדניום),โลหะโครโมลี,994,BB-7421
602,602,BB-8107,5.0,G,None,ML Bottom Bracket,44.9506,True,None,500,...,ML Bottom Bracket,\x474946383961f0009500f70000000000800000008000...,Aluminum alloy cups; large diameter spindle.,Cuvettes en alliage d'aluminium ; axe de grand...,铝合金车圈；大直径脚蹬轴。,فناجين من سبيكة الألومنيوم؛ ذات محور دوران كبي...,כיסויים מסגסוגת אלומיניום; ציר רחב-קוטר.,ดุมอลูมิเนียมอัลลอยด์ แกนเพลาขนาดใหญ่,995,BB-8107
603,604,BK-R19B-44,2.0,LB,CM,"Road-750 Black, 44",343.6496,True,Black,100,...,Road-750,\x474946383961f0009500f700008a7677f7f7f7edecec...,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,入门级成人自行车；确保越野旅行或公路骑乘的舒适。快拆式车毂和轮缘。,إنها دراجة مناسبة للمبتدئين من البالغين؛ فهي ت...,"אופני מבוגרים למתחילים; מציעים רכיבה נוחה ""מחו...",จักรยานระดับเริ่มต้นสำหรับผู้ใหญ่ ให้ความสบายใ...,997,BK-R19B-44
604,605,BK-R19B-48,2.0,LB,CM,"Road-750 Black, 48",343.6496,True,Black,100,...,Road-750,\x474946383961f0009500f700008a7677f7f7f7edecec...,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,入门级成人自行车；确保越野旅行或公路骑乘的舒适。快拆式车毂和轮缘。,إنها دراجة مناسبة للمبتدئين من البالغين؛ فهي ت...,"אופני מבוגרים למתחילים; מציעים רכיבה נוחה ""מחו...",จักรยานระดับเริ่มต้นสำหรับผู้ใหญ่ ให้ความสบายใ...,998,BK-R19B-48


In [19]:
sales_order = sales_order.merge(dim_product, left_on="ProductID", right_on="ProductID")

In [20]:
sales_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 66 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   SalesOrderID            60398 non-null  int64         
 1   RevisionNumber          60398 non-null  int64         
 2   OrderDate               60398 non-null  object        
 3   DueDate                 60398 non-null  object        
 4   ShipDate                60398 non-null  object        
 5   Status                  60398 non-null  int64         
 6   OnlineOrderFlag         60398 non-null  bool          
 7   SalesOrderNumber        60398 non-null  object        
 8   PurchaseOrderNumber     0 non-null      object        
 9   AccountNumber           60398 non-null  object        
 10  CustomerID              60398 non-null  int64         
 11  SalesPersonID           0 non-null      object        
 12  TerritoryID             60398 non-null  int64 

In [21]:
sales_order = sales_order.merge(dim_customer, left_on="CustomerID", right_on="CustomerKey")

In [22]:
sales_order = sales_order.merge(dim_promotion, left_on="SpecialOfferID", right_on="PromotionKey")

In [23]:
sales_order = sales_order.merge(dim_sales_territory, left_on="TerritoryID", right_on="SalesTerritoryKey")

In [24]:
sales_order = sales_order.merge(dim_dates.rename(columns={'DateKey':'OrderDateKey'}), left_on="OrderDate", right_on="FullDateAlternateKey")
sales_order = sales_order.merge(dim_dates.rename(columns={'DateKey':'DueDateKey'}), left_on="DueDate", right_on="FullDateAlternateKey")
sales_order = sales_order.merge(dim_dates.rename(columns={'DateKey':'ShipDateKey'}), left_on="ShipDate", right_on="FullDateAlternateKey")

In [25]:

sales_order.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 159 columns):
 #    Column                      Dtype         
---   ------                      -----         
 0    SalesOrderID                int64         
 1    RevisionNumber              int64         
 2    OrderDate                   object        
 3    DueDate                     object        
 4    ShipDate                    object        
 5    Status                      int64         
 6    OnlineOrderFlag             bool          
 7    SalesOrderNumber            object        
 8    PurchaseOrderNumber         object        
 9    AccountNumber               object        
 10   CustomerID                  int64         
 11   SalesPersonID               object        
 12   TerritoryID                 int64         
 13   BillToAddressID             int64         
 14   ShipToAddressID             int64         
 15   ShipMethodID                int64         
 16   Cr

In [26]:
sales_order = sales_order.rename(columns={
    "OrderQty":"OrderQuantity",
    "DiscountPct":"UnitPriceDiscountPct",
    "ToCurrencyCode":"CurrencyKey",
})

In [27]:
quantity_unitprice = zip(list(sales_order["OrderQuantity"]), list(sales_order["UnitPrice"]))
sales_order = sales_order.assign(ExtendedAmount=[x*y for x, y in quantity_unitprice])
sales_order

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,SpanishMonthName,FrenchMonthName,MonthNumberOfYear,CalendarQuarter,CalendarYear,CalendarSemester,FiscalQuarter,FiscalYear,FiscalSemester,ExtendedAmount
0,43697,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43697,None,10-4030-021768,...,Junio,Juin,6,2,2011,1,4,2011,2,3578.2700
1,43698,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43698,None,10-4030-028389,...,Junio,Juin,6,2,2011,1,4,2011,2,3399.9900
2,43699,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43699,None,10-4030-025863,...,Junio,Juin,6,2,2011,1,4,2011,2,3399.9900
3,43700,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43700,None,10-4030-014501,...,Junio,Juin,6,2,2011,1,4,2011,2,699.0982
4,43701,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43701,None,10-4030-011003,...,Junio,Juin,6,2,2011,1,4,2011,2,3399.9900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,75122,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75122,None,10-4030-015868,...,Julio,Juillet,7,3,2014,2,1,2015,1,21.9800
60394,75122,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75122,None,10-4030-015868,...,Julio,Juillet,7,3,2014,2,1,2015,1,8.9900
60395,75123,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75123,None,10-4030-018759,...,Julio,Juillet,7,3,2014,2,1,2015,1,21.9800
60396,75123,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75123,None,10-4030-018759,...,Julio,Juillet,7,3,2014,2,1,2015,1,159.0000


In [28]:
discountpct_discountamount = zip(list(sales_order["ExtendedAmount"]), list(sales_order["UnitPriceDiscountPct"]))
sales_order = sales_order.assign(DiscountAmount=[x*y for x, y in discountpct_discountamount])
sales_order

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,FrenchMonthName,MonthNumberOfYear,CalendarQuarter,CalendarYear,CalendarSemester,FiscalQuarter,FiscalYear,FiscalSemester,ExtendedAmount,DiscountAmount
0,43697,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43697,None,10-4030-021768,...,Juin,6,2,2011,1,4,2011,2,3578.2700,0.0
1,43698,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43698,None,10-4030-028389,...,Juin,6,2,2011,1,4,2011,2,3399.9900,0.0
2,43699,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43699,None,10-4030-025863,...,Juin,6,2,2011,1,4,2011,2,3399.9900,0.0
3,43700,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43700,None,10-4030-014501,...,Juin,6,2,2011,1,4,2011,2,699.0982,0.0
4,43701,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43701,None,10-4030-011003,...,Juin,6,2,2011,1,4,2011,2,3399.9900,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,75122,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75122,None,10-4030-015868,...,Juillet,7,3,2014,2,1,2015,1,21.9800,0.0
60394,75122,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75122,None,10-4030-015868,...,Juillet,7,3,2014,2,1,2015,1,8.9900,0.0
60395,75123,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75123,None,10-4030-018759,...,Juillet,7,3,2014,2,1,2015,1,21.9800,0.0
60396,75123,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75123,None,10-4030-018759,...,Juillet,7,3,2014,2,1,2015,1,159.0000,0.0


In [29]:
total_product_cost = zip(list(sales_order["OrderQuantity"]), list(sales_order["ProductStandardCost"]))
sales_order = sales_order.assign(TotalProductCost=[x*y for x, y in total_product_cost])
sales_order

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,MonthNumberOfYear,CalendarQuarter,CalendarYear,CalendarSemester,FiscalQuarter,FiscalYear,FiscalSemester,ExtendedAmount,DiscountAmount,TotalProductCost
0,43697,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43697,None,10-4030-021768,...,6,2,2011,1,4,2011,2,3578.2700,0.0,2171.2942
1,43698,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43698,None,10-4030-028389,...,6,2,2011,1,4,2011,2,3399.9900,0.0,1912.1544
2,43699,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43699,None,10-4030-025863,...,6,2,2011,1,4,2011,2,3399.9900,0.0,1912.1544
3,43700,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43700,None,10-4030-014501,...,6,2,2011,1,4,2011,2,699.0982,0.0,413.1463
4,43701,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43701,None,10-4030-011003,...,6,2,2011,1,4,2011,2,3399.9900,0.0,1912.1544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,75122,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75122,None,10-4030-015868,...,7,3,2014,2,1,2015,1,21.9800,0.0,8.2205
60394,75122,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75122,None,10-4030-015868,...,7,3,2014,2,1,2015,1,8.9900,0.0,6.9223
60395,75123,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75123,None,10-4030-018759,...,7,3,2014,2,1,2015,1,21.9800,0.0,8.2205
60396,75123,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75123,None,10-4030-018759,...,7,3,2014,2,1,2015,1,159.0000,0.0,59.4660


In [30]:
sales_order = sales_order.assign(SalesAmount=list(sales_order["ExtendedAmount"]))
sales_order

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,CalendarQuarter,CalendarYear,CalendarSemester,FiscalQuarter,FiscalYear,FiscalSemester,ExtendedAmount,DiscountAmount,TotalProductCost,SalesAmount
0,43697,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43697,None,10-4030-021768,...,2,2011,1,4,2011,2,3578.2700,0.0,2171.2942,3578.2700
1,43698,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43698,None,10-4030-028389,...,2,2011,1,4,2011,2,3399.9900,0.0,1912.1544,3399.9900
2,43699,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43699,None,10-4030-025863,...,2,2011,1,4,2011,2,3399.9900,0.0,1912.1544,3399.9900
3,43700,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43700,None,10-4030-014501,...,2,2011,1,4,2011,2,699.0982,0.0,413.1463,699.0982
4,43701,8,2011-05-31,2011-06-12,2011-06-07,5,True,SO43701,None,10-4030-011003,...,2,2011,1,4,2011,2,3399.9900,0.0,1912.1544,3399.9900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,75122,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75122,None,10-4030-015868,...,3,2014,2,1,2015,1,21.9800,0.0,8.2205,21.9800
60394,75122,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75122,None,10-4030-015868,...,3,2014,2,1,2015,1,8.9900,0.0,6.9223,8.9900
60395,75123,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75123,None,10-4030-018759,...,3,2014,2,1,2015,1,21.9800,0.0,8.2205,21.9800
60396,75123,8,2014-06-30,2014-07-12,2014-07-07,5,True,SO75123,None,10-4030-018759,...,3,2014,2,1,2015,1,159.0000,0.0,59.4660,159.0000


In [31]:
#product_standard_cost = pd.read_sql_query('SELECT * FROM "DimProduct";', db_etl)

In [32]:
result = sales_order[[
                    "ProductKey",
                    "OrderDateKey",
                      "DueDateKey",
                      "ShipDateKey",
                      "CustomerKey", "PromotionKey",
                       "CurrencyKey",
                      "SalesTerritoryKey",
                      "SalesOrderNumber",
                      #"SalesOrderLineNumber", #missing!,
                      "RevisionNumber",
                      "OrderQuantity",
                      "UnitPrice",
                      "ExtendedAmount",
                      "UnitPriceDiscountPct",
                      "DiscountAmount",
                      "ProductStandardCost",
                      "TotalProductCost",
                      "SalesAmount",
                      "TaxAmt",
                      "Freight",
                      "CarrierTrackingNumber",
                      #"CustomerPONumber", #missing!,
                      "OrderDate", "DueDate", "ShipDate"
                     ]]

In [33]:
result

,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,RevisionNumber,...,DiscountAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,OrderDate,DueDate,ShipDate
0,310,20110531,20110612,20110607,21768,1,19,6,SO43697,8,...,0.0,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,2011-05-31,2011-06-12,2011-06-07
1,346,20110531,20110612,20110607,28389,1,39,7,SO43698,8,...,0.0,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,2011-05-31,2011-06-12,2011-06-07
2,346,20110531,20110612,20110607,25863,1,100,1,SO43699,8,...,0.0,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,2011-05-31,2011-06-12,2011-06-07
3,336,20110531,20110612,20110607,14501,1,100,4,SO43700,8,...,0.0,413.1463,413.1463,699.0982,55.9279,17.4775,None,2011-05-31,2011-06-12,2011-06-07
4,346,20110531,20110612,20110607,11003,1,6,9,SO43701,8,...,0.0,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,2011-05-31,2011-06-12,2011-06-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,485,20140630,20140712,20140707,15868,1,100,6,SO75122,8,...,0.0,8.2205,8.2205,21.9800,2.4776,0.7743,None,2014-06-30,2014-07-12,2014-07-07
60394,223,20140630,20140712,20140707,15868,1,100,6,SO75122,8,...,0.0,6.9223,6.9223,8.9900,2.4776,0.7743,None,2014-06-30,2014-07-12,2014-07-07
60395,485,20140630,20140712,20140707,18759,1,100,6,SO75123,8,...,0.0,8.2205,8.2205,21.9800,15.1976,4.7493,None,2014-06-30,2014-07-12,2014-07-07
60396,486,20140630,20140712,20140707,18759,1,100,6,SO75123,8,...,0.0,59.4660,59.4660,159.0000,15.1976,4.7493,None,2014-06-30,2014-07-12,2014-07-07


## LOAD

In [34]:
db_op, db_etl = connect_databases()
result.to_sql('FactInternetSales', db_etl, if_exists='replace', index=False)

398